In [5]:
import os
os.chdir(r'f:\ML Lab Spring 25')

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [7]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [8]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [9]:
model1 = create_rnn()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 21)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 24, 8)          │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 20)             │           580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [24]:
checkpoints = 'F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = 'F:/ML Lab Spring 25/lab_9/checkpoints'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"/history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"/history.json"])

In [25]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
# TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [26]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [27]:
import os
path_dataset ='F:/ML Lab Spring 25/lab_9/dataset/'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

f:\ML Lab Spring 25\.venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [28]:
time_steps=24
num_features=21

In [29]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.326491355895996 sec


In [30]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1081 - mae: 0.1081 - mape: 2785.9927
Epoch 1: val_loss improved from inf to 0.01624, saving model to F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 48s 14ms/step - loss: 0.1080 - mae: 0.1080 - mape: 2784.2002 - val_loss: 0.0162 - val_mae: 0.0162 - val_mape: 7.8044
Epoch 2/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0166 - mae: 0.0166 - mape: 45.4050
Epoch 2: val_loss did not improve from 0.01624
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - loss: 0.0166 - mae: 0.0166 - mape: 45.4313 - val_loss: 0.0170 - val_mae: 0.0170 - val_mape: 7.3460
Epoch 3/60
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0142 - mae: 0.0142 - mape: 26.4817
Epoch 3: val_loss improved from 0.01624 to 0.01161, saving model to F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - loss: 0.0142 - mae: 0.0142 - mape: 26.7402 - val_loss: 0.0116 - val_mae: 0.0116 - val_mape: 5.6361
Epoch 4/60
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0136 - mae: 0.0136 - mape: 147.0341
Epoch 4: val_loss did not improve from 0.01161
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 53s 19ms/step - loss: 0.0136 - mae: 0.0136 - mape: 147.0027 - val_loss: 0.0136 - val_mae: 0.0136 - val_mape: 5.7745
Epoch 5/60
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0132 - mae: 0.0132 - mape: 71.7314
Epoch 5: val_loss improved from 0.01161 to 0.01156, saving model to F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 69s 14ms/step - loss: 0.0132 - mae: 0.0132 - mape: 71.7118 - val_loss: 0.0116 - val_mae: 0.0116 - val_mape: 5.0183
Epoch 6/60
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0125 - mae: 0.0125 - mape: 264.5952
Epoch 6: val_loss did not improve from 0.01156
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 40s 14ms/step - loss: 0.0125 - mae: 0.0125 - mape: 264.4186 - val_loss: 0.0136 - val_mae: 0.0136 - val_mape: 6.4342
Epoch 7/60
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0124 - mae: 0.0124 - mape: 201.6940
Epoch 7: val_loss did not improve from 0.01156
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 37s 14ms/step - loss: 0.0124 - mae: 0.0124 - mape: 201.6447 - val_loss: 0.0129 - val_mae: 0.0129 - val_mape: 5.5764
Epoch 8/60
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0120 - mae: 0.0120 - mape: 205.7676
Epoch 8: val_loss did not improve from 0.01156
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - loss: 0.0120 - mae: 0.0120 - mape: 205.6704 - val_loss: 0.0120 - val_

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 37s 14ms/step - loss: 0.0120 - mae: 0.0120 - mape: 17.9752 - val_loss: 0.0100 - val_mae: 0.0100 - val_mape: 4.1889
Epoch 10/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0116 - mae: 0.0116 - mape: 19.2711
Epoch 10: val_loss did not improve from 0.00997
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - loss: 0.0116 - mae: 0.0116 - mape: 19.2856 - val_loss: 0.0111 - val_mae: 0.0111 - val_mape: 5.5266
Epoch 11/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0115 - mae: 0.0115 - mape: 225.3447
Epoch 11: val_loss did not improve from 0.00997
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 38s 14ms/step - loss: 0.0115 - mae: 0.0115 - mape: 225.3810 - val_loss: 0.0132 - val_mae: 0.0132 - val_mape: 6.6221
Epoch 12/60
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0112 - mae: 0.0112 - mape: 100.0516
Epoch 12: val_loss did not improve from 0.00997
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 37s 14ms/step - loss: 0.0112 - mae: 0.0112 - mape: 100.0628 - val_loss: 0.0134 - 

...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - loss: 0.0084 - mae: 0.0084 - mape: 4.0657 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.1622
Epoch 42/60
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae: 0.0081 - mape: 192.3992
Epoch 42: val_loss did not improve from 0.00729
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 40s 9ms/step - loss: 0.0081 - mae: 0.0081 - mape: 192.2557 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.2720
Epoch 43/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0082 - mae: 0.0082 - mape: 149.7071
Epoch 43: val_loss did not improve from 0.00729
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - loss: 0.0082 - mae: 0.0082 - mape: 149.6980 - val_loss: 0.0085 - val_mae: 0.0085 - val_mape: 4.0579
Epoch 44/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0081 - mae: 0.0081 - mape: 163.4911
Epoch 44: val_loss did not improve from 0.00729
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0081 - mae: 0.0081 - mape: 163.1596 - val_loss: 0.0078 - val_m

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 29s 11ms/step - loss: 0.0080 - mae: 0.0080 - mape: 8.7949 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.2376
Epoch 46/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - mae: 0.0080 - mape: 2.6872
Epoch 46: val_loss did not improve from 0.00726
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0080 - mae: 0.0080 - mape: 2.6877 - val_loss: 0.0080 - val_mae: 0.0080 - val_mape: 3.5732
Epoch 47/60
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0080 - mae: 0.0080 - mape: 4.8748
Epoch 47: val_loss did not improve from 0.00726
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0080 - mae: 0.0080 - mape: 4.8777 - val_loss: 0.0102 - val_mae: 0.0102 - val_mape: 4.1785
Epoch 48/60
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0080 - mae: 0.0080 - mape: 130.1524
Epoch 48: val_loss did not improve from 0.00726
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0080 - mae: 0.0080 - mape: 130.0811 - val_loss: 0.0078 - val_mae: 0.00

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0080 - mae: 0.0080 - mape: 12.7510 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.0580
Epoch 54/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0080 - mae: 0.0080 - mape: 26.8921
Epoch 54: val_loss did not improve from 0.00725
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0080 - mae: 0.0080 - mape: 26.8961 - val_loss: 0.0083 - val_mae: 0.0083 - val_mape: 3.4539
Epoch 55/60
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0079 - mae: 0.0079 - mape: 13.1035
Epoch 55: val_loss did not improve from 0.00725
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - loss: 0.0079 - mae: 0.0079 - mape: 13.1504 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 3.1759
Epoch 56/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0079 - mae: 0.0079 - mape: 3.5306
Epoch 56: val_loss did not improve from 0.00725
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0079 - mae: 0.0079 - mape: 3.5546 - val_loss: 0.0100 - val_mae: 0.01

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 48s 11ms/step - loss: 0.0079 - mae: 0.0079 - mape: 34.1869 - val_loss: 0.0070 - val_mae: 0.0070 - val_mape: 2.9669
Epoch 59/60
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0079 - mae: 0.0079 - mape: 104.1168
Epoch 59: val_loss did not improve from 0.00701
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 43s 12ms/step - loss: 0.0079 - mae: 0.0079 - mape: 104.0760 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 3.6409
Epoch 60/60
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0078 - mae: 0.0078 - mape: 23.9614
Epoch 60: val_loss improved from 0.00701 to 0.00701, saving model to F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-0060-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 31s 11ms/step - loss: 0.0078 - mae: 0.0078 - mape: 24.0679 - val_loss: 0.0070 - val_mae: 0.0070 - val_mape: 3.2666


In [ ]:

model = load_model('F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-0053-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 2s 4ms/step
Mean Absolute Error (MAE): 110.66
Median Absolute Error (MedAE): 88.57
Mean Squared Error (MSE): 21191.35
Root Mean Squared Error (RMSE): 145.57
Mean Absolute Percentage Error (MAPE): 0.76 %
Median Absolute Percentage Error (MDAPE): 0.61 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [45]:
checkpoints = 'F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model_path = 'F:/ML Lab Spring 25/lab_9/checkpoints/E1-cp-0053-loss0.01.h5'
start_epoch = 53

In [ ]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
# TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= MeanAbsoluteError(), optimizer=opt, metrics=[MeanAbsoluteError(), MeanAbsolutePercentageError()])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

In [ ]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

In [ ]:

model = load_model('F:/ML Lab Spring 25/lab_9/checkpoints/E2-cp-0006-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 1ms/step
Mean Absolute Error (MAE): 144.75
Median Absolute Error (MedAE): 113.65
Mean Squared Error (MSE): 36189.68
Root Mean Squared Error (RMSE): 190.24
Mean Absolute Percentage Error (MAPE): 0.98 %
Median Absolute Percentage Error (MDAPE): 0.79 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
